- Sky wirte test code at 2024-06-18.
- The aim is to test the Hikrobot industrial camera functionality.

In [15]:
import ctypes
import numpy as np
import cv2
import time
import platform
from MvCameraControl_class import *

# 0 设备扫描与开启

In [16]:
cam = MvCamera()
device_list = MV_CC_DEVICE_INFO_LIST()
n_ret = MvCamera.MV_CC_EnumDevices(MV_GIGE_DEVICE | MV_USB_DEVICE, device_list)
print(n_ret)
device_count = device_list.nDeviceNum
print(device_count)

0
1


In [17]:
device_info_ptr = device_list.pDeviceInfo[0]
device_info = device_info_ptr.contents

if device_info.nTLayerType == MV_USB_DEVICE:
    chUserDefinedName = bytearray()
    for i in range(256):
        if device_info.SpecialInfo.stUsb3VInfo.chUserDefinedName[i] == 0:
            break
        chUserDefinedName.append(device_info.SpecialInfo.stUsb3VInfo.chUserDefinedName[i])
    if len(chUserDefinedName) == 0:
        chUserDefinedName = b"Not defined name by users"
    print(f"  设备名: {chUserDefinedName.decode()}")

  设备名: Not defined name by users


In [18]:
n_ret = cam.MV_CC_CreateHandle(device_info)
if n_ret != 0:
    print(f"[ERROR] 创建设备句柄失败！错误码: {n_ret:#x}")

n_ret = cam.MV_CC_OpenDevice(MV_ACCESS_Exclusive, 0)
if n_ret != MV_OK:
    print(f"[ERROR] 打开设备失败！错误码: {n_ret:#x}")

In [19]:
n_ret = cam.MV_CC_SetEnumValue("TriggerMode", MV_TRIGGER_MODE_OFF)
if n_ret != MV_OK:
    print(f"[WARNING] 设置触发模式失败: {n_ret:#x}")

try:
    # n_ret = cam.MV_CC_SetFloatValue("AcquisitionFrameRate", 30.0)
    n_ret = cam.MV_CC_SetBoolValue("AcquisitionFrameRateEnable", False)
    if n_ret == MV_OK:
        # print("  帧率设置为: 30.0 fps")
        print("  帧率自动控制已关闭")
except:
    pass

try:
    n_ret = cam.MV_CC_SetEnumValue("ExposureAuto", 2)  # 1 = 连续自动曝光
    cam.MV_CC_SetFloatValue("ExposureTimeUpper", 200000.0)
    if n_ret == MV_OK:
        print("  自动曝光已开启")
except:
    pass

try:
    # 将增益自动模式设置为连续（通常 0=Off, 1=Once, 2=Continuous）
    n_ret = cam.MV_CC_SetEnumValue("GainAuto", 2)
    cam.MV_CC_SetFloatValue("GainUpperLimit", 18.0)
    if n_ret == MV_OK:
        print("  自动增益已开启 (Continuous)")
    else:
        print(f"  [提示] 自动增益设置返回: {n_ret:#x}，上界为12.0")
except Exception as e:
    print(f"  设置自动增益时发生异常: {e}")

try:
    # Gamma值通常设为1.0（无校正）或2.2（sRGB标准，用于增强暗部细节）
    # 根据你的场景调整：物体颜色鲜艳且光照好可用1.0，光线较暗或需要提亮暗部可用2.2
    target_gamma = 1
    n_ret = cam.MV_CC_SetFloatValue("Gamma", target_gamma)
    if n_ret == MV_OK:
        print(f"  Gamma值已设置为: {target_gamma}")
    else:
        # 如果Gamma不支持，尝试设置GammaEnable
        n_ret = cam.MV_CC_SetBoolValue("GammaEnable", True)
        if n_ret == MV_OK:
            print("  Gamma校正已启用（使用默认曲线）")
except Exception as e:
    print(f"  设置Gamma时发生异常: {e}")

try:
    n_ret = cam.MV_CC_SetEnumValue("PixelFormat", PixelType_Gvsp_BayerRG8)
    if n_ret == MV_OK:
        print("  像素格式已设置为: Bayer RG 8")
    else:
        print(f"  [警告] 设置像素格式失败，错误码: {n_ret:#x}。将使用当前格式。")
except Exception as e:
    print(f"  设置像素格式时发生异常: {e}")

try:
    # 先尝试设置为自动白平衡（Continuous模式）
    n_ret = cam.MV_CC_SetEnumValue("BalanceWhiteAuto", 2)  # 2 = Continuous
    if n_ret == MV_OK:
        print("  自动白平衡已开启 (Continuous)")
    else:
        # 如果自动模式不支持，尝试设置手动白平衡系数（需根据场景调整）
        # 通常R和B通道需要调整，G通道保持1.0
        # cam.MV_CC_SetEnumValue("BalanceRatioSelector", 0)  # 选择Red通道
        # cam.MV_CC_SetFloatValue("BalanceRatio", 1.8)      # 设置Red系数
        # cam.MV_CC_SetEnumValue("BalanceRatioSelector", 2)  # 选择Blue通道
        # cam.MV_CC_SetFloatValue("BalanceRatio", 1.4)      # 设置Blue系数
        print("  自动白平衡不被支持，将使用默认或当前白平衡设置")
except Exception as e:
    print(f"  设置白平衡时发生异常: {e}")

try:
    # 启用锐化，提升边缘清晰度（有利于轮廓提取）
    n_ret = cam.MV_CC_SetBoolValue("SharpnessEnable", True)
    if n_ret == MV_OK:
        print("  锐化已启用")
except:
    pass

print("[INFO] 相机参数设置完成")

  帧率自动控制已关闭
  自动曝光已开启
  自动增益已开启 (Continuous)
  Gamma值已设置为: 1
  像素格式已设置为: Bayer RG 8
  自动白平衡已开启 (Continuous)
[INFO] 相机参数设置完成


# 1. 取流

In [20]:
n_ret = cam.MV_CC_StartGrabbing()
if n_ret != MV_OK:
    print(f"[ERROR] 开始采集失败！错误码: {n_ret:#x}")

## 1.1 单帧

In [21]:
# # 帧输出信息结构体
# stFrameInfo = MV_FRAME_OUT_INFO_EX()
# # 清空结构体
# memset(ctypes.byref(stFrameInfo), 0, ctypes.sizeof(stFrameInfo))
# # 创建缓冲区 (按最大可能尺寸：3072*2048*2， 应对12位非Packed的2字节/像素情况)
# data_buf_size = 3072 * 2048 * 2
# pData = (ctypes.c_ubyte * data_buf_size)()

# # 主动获取一帧图像 (超时时间1000ms)
# nRet = cam.MV_CC_GetOneFrameTimeout(pData, data_buf_size, stFrameInfo, 1000)
# if nRet != MV_OK:
#     print(f"获取图像失败！错误码: {nRet:#x}")
# else:
#     print(f"获取图像成功！宽：{stFrameInfo.nWidth}, 高：{stFrameInfo.nHeight}, 像素格式枚举值：{stFrameInfo.enPixelType:#x}")
    
#     # 将原始数据转换为 numpy 数组
#     image_array = np.frombuffer(pData, dtype=np.uint8, count=stFrameInfo.nFrameLen)
    
#     output_image = None # 用于存储最终转换后的BGR图像
#     # Bayer RG 8
#     if stFrameInfo.enPixelType == PixelType_Gvsp_BayerRG8:
#         print("  处理: Bayer RG 8 格式")
#         # 重塑为二维灰度图（每个像素点是一个Bayer颜色分量）
#         bayer_2d = image_array.reshape((stFrameInfo.nHeight, stFrameInfo.nWidth))
#         output_image = cv2.cvtColor(bayer_2d, cv2.COLOR_BayerRG2RGB)
    
#     else:
#         print(f"[ERROR] 无法处理的未知像素格式: {stFrameInfo.enPixelType:#x}")

#     if output_image is not None:
#         print(f"[SUCCESS] 图像转换成功，尺寸: {output_image.shape}")
#         hsv_image = cv2.cvtColor(output_image, cv2.COLOR_RGB2HSV)
        
#         # 定义红色的HSV阈值范围（红色在HSV色环上位于0°和180°附近，所以需要两个范围）
#         # 注意：这些阈值需要根据您的实际红色框和光照环境进行微调！
#         # H: 色相 (0-180), S: 饱和度 (0-255), V: 明度 (0-255)
#         lower_red1 = np.array([0, 100, 100])
#         upper_red1 = np.array([10, 255, 255])
#         lower_red2 = np.array([160, 100, 100])
#         upper_red2 = np.array([180, 255, 255])
        
#         # 根据阈值创建掩膜
#         mask_red1 = cv2.inRange(hsv_image, lower_red1, upper_red1)
#         mask_red2 = cv2.inRange(hsv_image, lower_red2, upper_red2)
#         red_mask = cv2.bitwise_or(mask_red1, mask_red2)
        
#         # 可选：进行形态学操作（如开运算、闭运算）去除噪声，连接断裂区域
#         kernel = np.ones((5,5), np.uint8)
#         red_mask_cleaned = cv2.morphologyEx(red_mask, cv2.MORPH_CLOSE, kernel)
#         red_mask_cleaned = cv2.morphologyEx(red_mask_cleaned, cv2.MORPH_OPEN, kernel)
        
#         # 在原始图像上显示掩膜，用于调试
#         cv2.imwrite('debug_red_mask.jpg', red_mask_cleaned)
        
#         # 在掩膜中查找轮廓
#         contours, _ = cv2.findContours(red_mask_cleaned, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
#         if contours:
#             # 假设最大的红色轮廓是我们的目标框
#             largest_contour = max(contours, key=cv2.contourArea)
            
#             # 计算轮廓的像素面积和外接矩形
#             pixel_area = cv2.contourArea(largest_contour)
#             x, y, w, h = cv2.boundingCircle(largest_contour)
            
#             # 在原始图像上绘制轮廓和外接矩形（用于可视化验证）
#             output_with_contour = output_image.copy()
#             cv2.drawContours(output_with_contour, [largest_contour], -1, (0, 255, 0), 3) # 绿色轮廓
#             cv2.rectangle(output_with_contour, (x, y), (x+w, y+h), (255, 0, 0), 2) # 蓝色外接框
#             cv2.imwrite('output_with_detection.jpg', output_with_contour)
            
#             # 计算轮廓的中心点像素坐标
#             center_x_pixel = x + w // 2
#             center_y_pixel = y + h // 2
#             cv2.circle(output_with_contour, (center_x_pixel, center_y_pixel), 10, (0, 0, 255), -1) # 红色中心点
#             cv2.imwrite('output_with_center.jpg', output_with_contour)
            
#             print(f"[检测成功] 红色轮廓像素中心: ({center_x_pixel}, {center_y_pixel})")
#             print(f"          外接矩形宽高: {w} x {h} 像素, 轮廓面积: {pixel_area:.1f} 像素")
            
#             KNOWN_DIAMETER = 0.02  # 单位：米 (m)
            
#             camera_matrix = np.array([
#                 [2500.0, 0,     1536.0],  # fx, 0,  cx
#                 [0,     2500.0, 1024.0],  # 0,  fy, cy
#                 [0,     0,     1.0]
#             ])
#             print(f"[提醒] 当前使用示例相机内参，请务必进行相机标定获取准确参数！")
            

#             focal_length_x = camera_matrix[0, 0]

#             r_pixel = np.sqrt(pixel_area / np.pi)
#             estimated_distance_z = (KNOWN_DIAMETER * focal_length_x) / (2 * r_pixel)
#             print(f"[三维估算] 方法2 - 基于圆形面积:")
#             print(f"          估算距离 (Z): {estimated_distance_z:.3f} 米")
            
#             cx = camera_matrix[0, 2]
#             cy = camera_matrix[1, 2]
#             fx = camera_matrix[0, 0]
#             fy = camera_matrix[1, 1]
            
#             estimated_x = (center_x_pixel - cx) * estimated_distance_z / fx
#             estimated_y = (center_y_pixel - cy) * estimated_distance_z / fy
            
#             print(f"[三维坐标] 红色框在相机坐标系下的位置:")
#             print(f"          X (水平): {estimated_x:.3f} 米")
#             print(f"          Y (垂直): {estimated_y:.3f} 米")
#             print(f"          Z (深度): {estimated_distance_z:.3f} 米")
            
#             hand_eye_matrix = np.eye(4) 
#             print(f"[提醒] 当前使用单位矩阵作为手眼变换矩阵，请务必进行手眼标定获取准确矩阵！")
            
#             # 构造相机坐标系下的点 (X, Y, Z, 1)
#             point_in_camera = np.array([estimated_x, estimated_y, estimated_distance_z, 1.0])
            
#             # 转换到机械臂基座坐标系
#             point_in_robot_base = hand_eye_matrix @ point_in_camera.T
            
#             print(f"[机械臂坐标] 红色框在机械臂基座坐标系下的位置:")
#             print(f"          X: {point_in_robot_base[0]:.3f} 米")
#             print(f"          Y: {point_in_robot_base[1]:.3f} 米")
#             print(f"          Z: {point_in_robot_base[2]:.3f} 米")
            
#         else:
#             print("[检测失败] 未找到红色轮廓，请检查HSV阈值或光照条件。")
#         # cv2.imshow('Converted Image', output_image)
#         # cv2.waitKey(1)
#         cv2.imwrite('first_frame_debug.jpg', output_image)
#     else:
#         print("[ERROR] 图像转换失败，无法得到输出图像。")

## 1.2 连续取流
- 相机本身无焦距参数，通过C-mout镜头

In [22]:
nFocalLength = ctypes.c_int()
n_ret = cam.MV_CC_GetIntValue("FocalLength", nFocalLength)
print(f"相机焦距: {nFocalLength.value} 微米")


相机焦距: 0 微米


In [23]:
ARM_X = 9*25 # mm
ARM_Z = - 27*25 # mm

In [24]:
CAMERA_FOCAL_LENGTH = 6.0
# 像元尺寸（从相机参数获取）
PIXEL_SIZE = 0.0024
# 实际小球直径
BALL_ACTUAL_DIAMETER = 120.0  # mm
# 相机分辨率（从相机参数获取）
CAMERA_RES_WIDTH = 3072
CAMERA_RES_HEIGHT = 2048
frame_captured = False

In [25]:
def analyze_ball_frame(image, mask, contours, frame_width, frame_height):
    """分析截取帧，计算小球的x,y,z坐标（相机坐标系）"""
    if not contours:
        print("[ERROR] 截取帧中未检测到红色小球！")
        return None
    
    # 1. 找到最大轮廓（假设是红色小球）
    largest_contour = max(contours, key=cv2.contourArea)
    
    # 2. 计算边界框和中心点（像素坐标）
    x, y, w, h = cv2.boundingRect(largest_contour)
    cx_pixel = x + w // 2
    cy_pixel = y + h // 2
    
    
    
    # 4. 计算小球在图像中的像素直径（使用平均直径）
    ball_pixel_diameter = (w + h) / 2.0
    # ball_pixel_diameter = max(w, h)
    
    if ball_pixel_diameter == 0:
        print("[ERROR] 小球像素直径为0，无法计算深度！")
        return None
    
    # 5. 使用相似三角形原理计算深度（Z坐标，单位：mm）
    # 公式：深度 = (实际直径 × 焦距) / (像素直径 × 像素尺寸)
    depth_mm = (BALL_ACTUAL_DIAMETER * CAMERA_FOCAL_LENGTH) / (ball_pixel_diameter * PIXEL_SIZE)
    
    # 3. 计算物理平面坐标（以图像中心为原点，单位：mm）
    # X轴向右为正，Y轴向下为正（相机坐标系）
    cx_physical = (cx_pixel - frame_width/2) * depth_mm * PIXEL_SIZE / CAMERA_FOCAL_LENGTH
    cy_physical = (cy_pixel - frame_height/2) * depth_mm * PIXEL_SIZE / CAMERA_FOCAL_LENGTH
    # 6. 创建3D坐标向量 [X, Y, Z]
    # 注意：这是相机坐标系下的坐标，原点在相机光心
    # X轴向右，Y轴向下，Z轴指向拍摄方向（正前方）
    ball_3d_coords = {
        'X': cx_physical,      # 单位：mm
        'Y': cy_physical,      # 单位：mm  
        'Z': depth_mm,         # 单位：mm
        'pixel_X': cx_pixel,   # 像素坐标
        'pixel_Y': cy_pixel,   # 像素坐标
        'pixel_diameter': ball_pixel_diameter
    }
    
    relative_x = depth_mm-ARM_Z
    relative_y = -(cx_physical-ARM_X)
    # 7. 输出结果
    print("="*50)
    print("【小球3D坐标分析结果】")
    print(f"1. 像素坐标 (X, Y)：({cx_pixel}, {cy_pixel})")
    print(f"{w} {h}")
    print(f"2. 物理坐标 (X, Y)：({cx_physical:.2f} mm, {cy_physical:.2f} mm)")
    print(f"3. 小球像素直径：{ball_pixel_diameter:.2f} 像素")
    print(f"4. 深度（Z坐标）：{depth_mm:.2f} mm")
    print(f"5. 3D坐标 (X,Y,Z)：({cx_physical:.2f}, {cy_physical:.2f}, {depth_mm:.2f}) mm")
    print(f"相对机械臂坐标：X=({relative_x}, Y=({relative_y}))")
    print("="*50)
    
    # 8. 在图像上标注结果
    cv2.putText(image, f"Center: ({cx_pixel}, {cy_pixel})", (10, 30), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 0), 2)
    cv2.putText(image, f"3D: ({cx_physical:.1f}, {cy_physical:.1f}, {depth_mm:.1f}) mm", 
                (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 0), 2)
    cv2.putText(image, f"Diameter: {ball_pixel_diameter:.1f} px", (10, 110), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 0), 2)
    
    # 9. 绘制中心点和轮廓
    cv2.circle(image, (cx_pixel, cy_pixel), 5, (0, 255, 255), -1)  # 黄色中心点
    cv2.drawContours(image, [largest_contour], -1, (0, 255, 0), 2)  # 绿色轮廓
    
    # 10. 保存结果图像
    cv2.imwrite("ball_analysis_result.png", image)
    cv2.imwrite("ball_mask.png", mask)
    print("✅ 分析结果已保存：ball_analysis_result.png / ball_mask.png")
    
    return ball_3d_coords

In [26]:
# 创建帧信息结构体
stFrameInfo = MV_FRAME_OUT_INFO_EX()
# 内存初始化为0
memset(ctypes.byref(stFrameInfo), 0, ctypes.sizeof(stFrameInfo))

# 创建缓冲区
data_buf_size = 3072 * 2048 * 2
pData = (ctypes.c_ubyte * data_buf_size)()

while True:
    nRet = cam.MV_CC_GetOneFrameTimeout(
        pData,
        data_buf_size,
        stFrameInfo,
        1000 # 等待时间
    )

    if nRet != MV_OK:
        print(f"[WARN] 获取图像失败: {nRet:#x}")
        continue

    # numpy
    image_array = np.frombuffer(
        pData,
        dtype=np.uint8,
        count=stFrameInfo.nFrameLen
    )

    output_image = None

    # Bayer RG8
    if stFrameInfo.enPixelType == PixelType_Gvsp_BayerRG8:
        bayer_2d = image_array.reshape(
            (stFrameInfo.nHeight, stFrameInfo.nWidth)
        )
        output_image = cv2.cvtColor(
            bayer_2d,
            cv2.COLOR_BAYER_RG2RGB

        )
    else:
        print(f"[ERROR] 不支持的像素格式: {stFrameInfo.enPixelType:#x}")
        continue

    # ================== 红色检测 ==================
    hsv_image = cv2.cvtColor(output_image, cv2.COLOR_BGR2HSV)

    lower_red1 = np.array([0, 100, 100])
    upper_red1 = np.array([10, 255, 255])
    lower_red2 = np.array([160, 100, 100])
    upper_red2 = np.array([180, 255, 255])

    mask1 = cv2.inRange(hsv_image, lower_red1, upper_red1)
    mask2 = cv2.inRange(hsv_image, lower_red2, upper_red2)
    red_mask = cv2.bitwise_or(mask1, mask2)

    kernel = np.ones((5, 5), np.uint8)
    red_mask = cv2.morphologyEx(red_mask, cv2.MORPH_CLOSE, kernel)
    red_mask = cv2.morphologyEx(red_mask, cv2.MORPH_OPEN, kernel)

    contours, _ = cv2.findContours(
        red_mask,
        cv2.RETR_EXTERNAL,
        cv2.CHAIN_APPROX_SIMPLE
    )

    if contours:
        largest = max(contours, key=cv2.contourArea)
        x, y, w, h = cv2.boundingRect(largest)

        cv2.rectangle(
            output_image,
            (x, y),
            (x + w, y + h),
            (0, 255, 0),
            2
        )

        cx = x + w // 2
        cy = y + h // 2
        cv2.circle(output_image, (cx, cy), 6, (0, 0, 255), -1)

    # ================== 显示 ==================
    cv2.namedWindow("Camera Stream", cv2.WINDOW_NORMAL | cv2.WINDOW_KEEPRATIO)
    cv2.namedWindow("Red Mask", cv2.WINDOW_NORMAL | cv2.WINDOW_KEEPRATIO)

    # # 2. 可选：设置窗口初始大小（比如先缩到800x600，可根据需求调整）
    # cv2.resizeWindow("Camera Stream", 800, 600)
    # cv2.resizeWindow("Red Mask", 800, 600)

    # 3. 显示图像（此时窗口已支持手动调整大小）
    cv2.imshow("Camera Stream", output_image)
    cv2.imshow("Red Mask", red_mask)

    key = cv2.waitKey(1) & 0xFF
    if key == ord('q') or key == 27:  # q 或 ESC
        break
    elif key == 13 and not frame_captured:  # Enter 键截取稳定帧
        print("📸 开始分析稳定帧...")
        # 调用分析函数
        analyze_ball_frame(
            output_image.copy(),  # 传副本避免实时帧覆盖
            red_mask.copy(),
            contours,
            stFrameInfo.nWidth,   # 图像实际宽度
            stFrameInfo.nHeight   # 图像实际高度
        )
        frame_captured = True
        print("✅ 帧分析完成！继续按q/ESC退出。")
    frame_captured = False

📸 开始分析稳定帧...
【小球3D坐标分析结果】
1. 像素坐标 (X, Y)：(2536, 1255)
651 461
2. 物理坐标 (X, Y)：(215.83 mm, 49.86 mm)
3. 小球像素直径：556.00 像素
4. 深度（Z坐标）：539.57 mm
5. 3D坐标 (X,Y,Z)：(215.83, 49.86, 539.57) mm
相对机械臂坐标：X=(1214.5683453237411, Y=(9.172661870503589))
✅ 分析结果已保存：ball_analysis_result.png / ball_mask.png
✅ 帧分析完成！继续按q/ESC退出。
📸 开始分析稳定帧...
【小球3D坐标分析结果】
1. 像素坐标 (X, Y)：(2679, 1279)
742 543
2. 物理坐标 (X, Y)：(213.48 mm, 47.63 mm)
3. 小球像素直径：642.50 像素
4. 深度（Z坐标）：466.93 mm
5. 3D坐标 (X,Y,Z)：(213.48, 47.63, 466.93) mm
相对机械臂坐标：X=(1141.9260700389104, Y=(11.521400778210136))
✅ 分析结果已保存：ball_analysis_result.png / ball_mask.png
✅ 帧分析完成！继续按q/ESC退出。
📸 开始分析稳定帧...
【小球3D坐标分析结果】
1. 像素坐标 (X, Y)：(1869, 1262)
657 460
2. 物理坐标 (X, Y)：(71.55 mm, 51.14 mm)
3. 小球像素直径：558.50 像素
4. 深度（Z坐标）：537.15 mm
5. 3D坐标 (X,Y,Z)：(71.55, 51.14, 537.15) mm
相对机械臂坐标：X=(1212.1530886302598, Y=(153.45120859444938))
✅ 分析结果已保存：ball_analysis_result.png / ball_mask.png
✅ 帧分析完成！继续按q/ESC退出。
📸 开始分析稳定帧...
【小球3D坐标分析结果】
1. 像素坐标 (X, Y)：(1869, 1260)
653 463
2. 物理坐标 (X, Y)

In [27]:
cv2.destroyAllWindows()

# 2. 结束处理

In [28]:
n_ret = cam.MV_CC_StopGrabbing()
if n_ret != MV_OK:
    print(f"[ERROR] 结束采集失败！错误码: {n_ret:#x}")

n_ret = cam.MV_CC_CloseDevice()
if n_ret != MV_OK:
    print(f"[ERROR] 关闭设备失败！错误码: {n_ret:#x}")
    
n_ret = cam.MV_CC_DestroyHandle()
if n_ret != MV_OK:
    print(f"[ERROR] 摧毁句柄失败！错误码: {n_ret:#x}")